```{contents}
```
## **Manual Intervention in LangGraph**

**Manual Intervention** in LangGraph refers to the deliberate insertion of **human control** into an otherwise automated execution graph.
It enables humans to **pause, inspect, modify, approve, or redirect** the graph’s execution by manipulating the **shared state** before the system proceeds.

This mechanism is essential for **safety, reliability, compliance, and production governance**.

---

### **1. Why Manual Intervention Is Necessary**

Pure automation is unsafe in many real systems.

| Risk                  | Why Manual Intervention Is Needed   |
| --------------------- | ----------------------------------- |
| Hallucinations        | Humans correct wrong outputs        |
| Regulatory compliance | Legal approval required             |
| High-risk actions     | Prevent irreversible damage         |
| Model drift           | Human oversight stabilizes behavior |
| Business rules        | Enforce domain constraints          |

---

### **2. Conceptual Model**

LangGraph models intervention as a **controlled execution interrupt**.

```
Automated Flow → [ Interrupt ] → Human Review → State Update → Resume
```

The graph is not broken; it is **paused at a safe checkpoint**.

---

### **3. Where Intervention Occurs**

Intervention is implemented through **special nodes**:

| Node Type       | Purpose                   |
| --------------- | ------------------------- |
| Interrupt Node  | Pause execution           |
| Approval Node   | Require explicit approval |
| Review Node     | Allow edits               |
| Correction Node | Inject fixes              |
| Override Node   | Replace state values      |

---

### **4. State-Based Intervention Mechanism**

LangGraph intervention always operates on **state**, not on code.

```python
class State(TypedDict):
    draft: str
    approved: bool
```

The graph flow depends on state transitions.

---

### **5. Minimal Example: Approval Gate**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    text: str
    approved: bool

def generate(state):
    return {"text": "Generated report", "approved": False}

def human_review(state):
    print("Review:", state["text"])
    decision = input("Approve? (y/n): ")
    return {"approved": decision == "y"}

def router(state):
    return END if state["approved"] else "human_review"

builder = StateGraph(State)
builder.add_node("generate", generate)
builder.add_node("human_review", human_review)

builder.set_entry_point("generate")
builder.add_edge("generate", "human_review")
builder.add_conditional_edges("human_review", router, {
    END: END,
    "human_review": "human_review"
})

graph = builder.compile()
graph.invoke({})
```

**Behavior**

* LLM generates content
* Human reviews it
* If rejected, execution loops
* If approved, graph completes

---

### **6. Variants of Manual Intervention**

| Variant              | Purpose                       |
| -------------------- | ----------------------------- |
| Approval Gate        | Require permission to proceed |
| Correction Injection | Human edits output            |
| State Override       | Replace incorrect state       |
| Execution Halt       | Stop the graph                |
| Branch Override      | Force alternate path          |
| Escalation Gate      | Transfer to expert            |
| Rollback             | Restore previous checkpoint   |

---

### **7. Production Use Cases**

| Domain        | Use                  |
| ------------- | -------------------- |
| Finance       | Trade approval       |
| Healthcare    | Diagnosis validation |
| Legal         | Contract review      |
| Cybersecurity | Incident response    |
| DevOps        | Deployment approval  |

---

### **8. Safety & Compliance Properties**

| Property       | Guarantee                 |
| -------------- | ------------------------- |
| Auditability   | Every intervention logged |
| Reversibility  | Rollback supported        |
| Traceability   | Full execution history    |
| Non-bypassable | Cannot skip intervention  |
| Governed       | Policy enforced           |

---

### **9. Why This Matters Architecturally**

Manual intervention converts an LLM pipeline into a **controlled decision system**.

Without intervention:
→ brittle automation

With intervention:
→ **governed autonomous system**

---

### **10. Mental Model**

> LangGraph treats humans as **first-class computational nodes**.

They are simply another **state transformer** in the graph.


### Demonstration

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# -----------------------------
# 1. Define Shared State
# -----------------------------
class State(TypedDict):
    draft: str
    approved: bool

# -----------------------------
# 2. Automated Generation Node
# -----------------------------
def generate(state: State) -> State:
    return {
        "draft": "AI generated business proposal v1",
        "approved": False
    }

# -----------------------------
# 3. Human Intervention Node
# -----------------------------
def human_review(state: State) -> State:
    print("\n--- HUMAN REVIEW ---")
    print("Draft:", state["draft"])
    
    choice = input("Approve (a) | Edit (e) | Reject (r): ").lower()

    if choice == "e":
        new_text = input("Enter corrected draft: ")
        return {"draft": new_text, "approved": False}

    if choice == "a":
        return {"approved": True}

    return {"approved": False}

# -----------------------------
# 4. Routing Logic
# -----------------------------
def router(state: State):
    if state["approved"]:
        return END
    return "human_review"

# -----------------------------
# 5. Build Graph
# -----------------------------
builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("human_review", human_review)

builder.set_entry_point("generate")
builder.add_edge("generate", "human_review")
builder.add_conditional_edges("human_review", router, {
    "human_review": "human_review",
    END: END
})

graph = builder.compile()

# -----------------------------
# 6. Execute Graph
# -----------------------------
result = graph.invoke({})
print("\nFINAL RESULT:", result)



--- HUMAN REVIEW ---
Draft: AI generated business proposal v1


Approve (a) | Edit (e) | Reject (r):  a



FINAL RESULT: {'draft': 'AI generated business proposal v1', 'approved': True}
